In [ ]:
# class DMC_Sampler(Sampler):
#     def __init__(self, SUR_data) -> None:
#         super().__init__(SUR_data)
        
#     def sample(self):
#         pass

In [1]:
from utils import generate_SUR, Sampler, DefaultParams
import numpy as np
from numpy.linalg import inv
from scipy.stats import invwishart, invgamma


In [3]:
data = generate_SUR()
params = DefaultParams()

In [8]:
z1 = data['x1']
z2 = np.concatenate((data['x2'], data['U'][:params.n]), axis=1)

In [10]:
y1 = data['Y'][:100]
y2 = data['Y'][100:]
b1_hat = inv(np.transpose(z1) @ z1) @ np.transpose(z1) @ y1
b2_hat = inv(np.transpose(z2) @ z2) @ np.transpose(z2) @ y2

In [16]:
N = 1000

In [26]:
a1 = (params.n - 2) / 2
scale1 = (np.transpose(y1 - z1 @ b1_hat) @ (y1 - z1 @ b1_hat)).item() / 2
sigma1 = [invgamma.rvs(a=a1, scale=scale1) for _ in range(N)]

In [20]:
mean1 = b1_hat.reshape(-1,)
b1 = [np.random.multivariate_normal(mean=mean1, cov=inv(np.transpose(z1) @ z1)*sig1).reshape(-1,1) for sig1 in sigma1]

In [25]:
a2 = (params.n - 1) / 2
scale2 = (np.transpose(y2 - z2 @ b2_hat) @ (y2 - z2 @ b2_hat)).item() / 2
sigma2 = [invgamma.rvs(a=a2, scale=scale2) for _ in range(N)]

In [28]:
mean2 = b2_hat.reshape(-1,)
b2 = [np.random.multivariate_normal(mean=mean2, cov=inv(np.transpose(z2) @ z2)*sig2).reshape(-1,1) for sig2 in sigma2]

In [32]:
rho = [ele[-1].item() for ele in b2]

In [34]:
w11 = sigma1
w12 = [rho_ele * sigma1_ele + sigma2_ele for (rho_ele, sigma1_ele, sigma2_ele) in zip(rho, sigma1, sigma2)]
w22 = [rho_ele * sigma1_ele for (rho_ele, sigma1_ele) in zip(rho, sigma1)]

In [37]:
Omega = [np.array([[w11_ele, w12_ele], 
                   [w12_ele, w22_ele]]) for (w11_ele, w12_ele, w22_ele) in zip(w11, w12, w22)]

In [38]:
X = data['X']
Y = data['Y']
X_transposed = np.transpose(X)

In [ ]:
# beta_hat = [inv(X_transposed @ inv(omega_ele) @ X) @ X_transposed @ inv(omega_ele) @ Y for omega_ele in Omega]
# omege_hat = [inv(X_transposed @ inv(omega_ele) @ X) for omega_ele in Omega]

